# Lab 2: Characteristics of remotely sensed data

In [17]:
from datetime import date
today = date.today()
d2 = today.strftime("%B %d, %Y")
print("Updated by Alfonso Torres-Rua, ", d2)

Updated by Alfonso Torres-Rua,  February 20, 2024


## **Purpose:**
The purpose of this lab is to demonstrate concepts of spatial, spectral, temporal and radiometric resolution.  You will be introduced to image data from several sensors aboard various platforms.  At the completion of the lab, you will be able to understand the difference between remotely sensed datasets based on sensor characteristics and how to choose an appropriate remotely sensed dataset based on these concepts.  

**Prerequisites:** Lab 1

## 1. Temporal Resolution

Every satellite is different from each other, spectrally, radiometrically, and temporally. This is due to multiple reasons such as access to technology, monetary resources, application targets, mission duration, and others. See these videos for two satellites:

- Landsat orbit:
https://svs.gsfc.nasa.gov//vis/a010000/a011400/a011481/G2014-016_LDCM_orbit_MASTER_ipod_lg.m4v

- MODIS orbit: https://svs.gsfc.nasa.gov//vis/a000000/a003300/a003348/aquacomp720.mp4

## 2. Spatial resolution

In the present context, spatial resolution means pixel size.  In practice, spatial resolution depends on the projection of the sensor's instantaneous field of view (IFOV) on the ground and how a set of radiometric measurements are resampled into a regular grid.  To see the difference in spatial resolution resulting from different sensors, visualize data at different scales from different sensors.

In [18]:
import ee
ee.Initialize()

import geemap

Map = geemap.Map() 
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

### - MODIS.  
There are two Moderate Resolution Imaging Spectro-Radiometers ([MODIS](http://modis.gsfc.nasa.gov/)) aboard the [Terra](http://terra.nasa.gov/) and [Aqua](http://aqua.nasa.gov/) satellites.  Different MODIS [bands](http://modis.gsfc.nasa.gov/about/specifications.php) produce data at different spatial resolutions.  For the visible bands, the lowest common resolution is 500 meters (red and NIR are 250 meters).  

Search for 'MYD09GA' in the [Google EarthEngine datasets](https://developers.google.com/earth-engine/datasets/catalog/) and retrieve (copy) the path to 'MYD09GA.006 Aqua Surface Reflectance Daily Global 1km and 500m'.  Name the import myd09. ([Complete list of MODIS land products](https://lpdaac.usgs.gov/dataset_discovery/modis/modis_products_table).  Note that Terra MODIS datasets start with 'MOD' and MODIS Aqua datasets start with 'MYD').

In [19]:
myd09 = ee.ImageCollection("MODIS/061/MCD43A4")


latitude =37.77862975352838
longitude = -122.22365888039644
sfoPoint = ee.Geometry.Point([longitude, latitude]);



To display a false-color MODIS image, select an image acquired by the Aqua MODIS sensor and display it for SFO:

In [20]:
# Get a surface reflectance image from the MODIS MYD09GA collection.
modisImage = ee.Image(myd09.filterDate('2023-08-05').first());

modisImage = modisImage.multiply(0.0001)
# Print the image band names to the console.
band_names = modisImage.bandNames()
print(band_names.getInfo())

['Nadir_Reflectance_Band1', 'Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band3', 'Nadir_Reflectance_Band4', 'Nadir_Reflectance_Band5', 'Nadir_Reflectance_Band6', 'Nadir_Reflectance_Band7', 'BRDF_Albedo_Band_Mandatory_Quality_Band1', 'BRDF_Albedo_Band_Mandatory_Quality_Band2', 'BRDF_Albedo_Band_Mandatory_Quality_Band3', 'BRDF_Albedo_Band_Mandatory_Quality_Band4', 'BRDF_Albedo_Band_Mandatory_Quality_Band5', 'BRDF_Albedo_Band_Mandatory_Quality_Band6', 'BRDF_Albedo_Band_Mandatory_Quality_Band7']


In [22]:
# Use these MODIS bands for red, green, blue, respectively.
modisBands = ['Nadir_Reflectance_Band1', 'Nadir_Reflectance_Band4', 'Nadir_Reflectance_Band3'];

#// Add the image to the map.
# url = modisImage.getThumbURL({'dimensions': 700,'bands':modisBands,'min': 0, 'max': 3000,'region':area_to_display})
# # print(url)
# Image(url=url)

# add layer
Map.addLayer(modisImage, {'bands':modisBands,'min': 0, 'max':0.4}, name='MODIS Image')
Map

Map(bottom=754.0, center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

These names are described also here: [modis bands](https://modis.gsfc.nasa.gov/about/specifications.php)

Note the size of pixels with respect to objects on the ground.  Print the size of the pixels (in meters) with:


In [23]:
# Get the scale of the data from the first band's projection:
modisScale = modisImage.select('Nadir_Reflectance_Band4').projection().nominalScale();

print('MODIS scale:', modisScale.getInfo());


MODIS scale: 463.31271652791656


In [24]:
# add layer SensorZenith
# Map.addLayer(modisImage, {'bands':'SensorZenith','min': 0, 'max': 0.3}, name='MODIS SensorZenith')

It's also worth noting that these MYD09 data are surface reflectance scaled by 10000 (not TOA reflectance), meaning that clever NASA scientists have done a fancy atmospheric correction for you!


## - Landsat MSS.  

Multi-spectral scanners (MSS) were flown aboard Landsats 1-5.  MSS data have a spatial resolution of 60 meters.
Search for 'landsat 5 mss' and import the result called 'USGS Landsat 5 MSS Collection 1 Tier 2 Raw Scenes'.  Name the import mss.

To visualize MSS data over SFO, for a relatively less cloudy image, use:


In [25]:
# Filter MSS imagery by location, date and cloudiness.
mss= ee.ImageCollection("LANDSAT/LM01/C02/T1")

mssImage = ee.Image(mss
    .filterBounds(sfoPoint)
    .sort('CLOUD_COVER')
#Get the least cloudy image.
    .first());




# Print the image band names to the console.
band_names = mssImage.bandNames()
print(band_names.getInfo())
# Display the MSS image as a color-IR composite.


['B4', 'B5', 'B6', 'B7', 'QA_PIXEL', 'QA_RADSAT']


In [26]:
# Use these MODIS bands for red, green, blue, respectively.
mssBands = ['B6', 'B5', 'B4'];

Map.addLayer(mssImage, {'bands':mssBands,'min': 0, 'max': 70}, name='LandsatMSS Image')
Map
# bounds = sfoPoint.buffer(70000)
# Map.centerObject(bounds)

Map(bottom=17798.0, center=[-12.146745814539685, -56.8048095703125], controls=(WidgetControl(options=['positio…

Check the scale (in meters) as previously:

In [27]:
# Get the scale of the MSS data from its projection:
mssScale = mssImage.select('B5').projection().nominalScale();

print('MSS scale:', mssScale.getInfo());


MSS scale: 60


## -TM.  

The Thematic Mapper ([TM](http://landsat.gsfc.nasa.gov/?p=3229)) was flown aboard Landsats 4-5.  (It was succeeded by the Enhanced Thematic Mapper ([ETM+](http://landsat.gsfc.nasa.gov/?p=3225)) aboard Landsat 7 and the Operational Land Imager ([OLI](http://landsat.gsfc.nasa.gov/?p=5447)) / Thermal Infrared Sensor ([TIRS](http://landsat.gsfc.nasa.gov/?p=5474)) sensors aboard Landsat 8.)  TM data have a spatial resolution of 30 meters.

1. Search for 'landsat 5 toa' and import the first result (which should be 'USGS Landsat 5 TM Collection 1 Tier 1 TOA Reflectance'.  Name the import tm.

2. Visualize MSS data over SFO, for approximately the same area as the MSS image, use  image.geometry():


In [28]:
tm= ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")

# // Filter TM imagery by location, date and cloudiness.
tmImage = ee.Image(tm
    .filterBounds(sfoPoint) # I could have used sfopoint too. every image has a boundary geometry
    .filterDate('2011-06-01', '2011-07-01')
    .sort('CLOUD_COVER') # not all satellites have this property, check the properties table
    .first());

tmImage = tmImage.multiply(2.75e-05).add(-0.2)


# Print the image band names to the console.
band_names = tmImage.bandNames()
print(band_names.getInfo())

['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'SR_ATMOS_OPACITY', 'SR_CLOUD_QA', 'ST_B6', 'ST_ATRAN', 'ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_QA', 'ST_TRAD', 'ST_URAD', 'QA_PIXEL', 'QA_RADSAT']


In [29]:
# // Display the TM image as a color-IR composite.
tmBands=  ['SR_B3', 'SR_B2', 'SR_B1']

Map.addLayer(tmImage, {'bands':tmBands,'min': 0, 'max': 0.4}, name='Landsat TM Image')
Map
# bounds = sfoPoint.buffer(70000)
# Map.centerObject(bounds)

Map(bottom=25695.0, center=[37.496652341233364, -123.13751220703126], controls=(WidgetControl(options=['positi…

Check the scale (in meters) as previously:

In [30]:
# // Get the scale of the TM data from its projection:
tmScale = tmImage.select('SR_B1').projection().nominalScale();
print('TM scale:', tmScale.getInfo());


TM scale: 30


In [13]:
## - Sentinel 2 

## -NAIP.  
The National Agriculture Imagery Program ([NAIP](http://www.fsa.usda.gov/programs-and-services/aerial-photography/imagery-programs/naip-imagery/)) is an effort to acquire imagery over the continental US on a 3-year rotation using airborne sensors.  The imagery have a spatial resolution of 1-2 meters.  

1. Search for 'naip' and import the first result (which should be 'NAIP: National Agriculture Imagery Program'.  Name the import naip.

2. Since NAIP imagery is distributed as quarters of Digital Ortho Quads (DOQs) at irregular cadence, load everything from the most recent year in the acquisition cycle (2012) over the study area and mosaic() it:


In [33]:
naip = ee.ImageCollection("USDA/NAIP/DOQQ")

# // Get NAIP images for the study period and region of interest.
naipImages = naip.filterDate('2019-01-01', '2023-12-31').filterBounds(sfoPoint);

# // Mosaic adjacent images into a single image.
naipImage = naipImages.mosaic();

# Print the image band names to the console.
band_names = naipImage.bandNames()
print(band_names.getInfo())


['R', 'G', 'B', 'N']


In [34]:
# // Display the NAIP mosaic as a color-IR composite.
nBands=  ['R', 'G', 'B']

Map.addLayer(naipImage, {'bands':nBands,'min': 0, 'max': 255}, name='NAIP Image')
Map
# bounds = sfoPoint.buffer(70000)
# Map.centerObject(bounds)

Map(bottom=1621000.0, center=[37.80788523279169, -122.27641582489015], controls=(WidgetControl(options=['posit…

In [35]:
# // Get the NAIP resolution from the first image in the mosaic.
naipScale = ee.Image(naipImages.first()).projection().nominalScale();
print('NAIP scale:', naipScale.getInfo());


NAIP scale: 0.6


## Assignment
1. What are the necessary changes to this notebook, to perform the same steps for Cache Valley? Produce a notebook.

2. There are more satellites and data sources available to you. To your Logan notebook add the following satellites completing the same steps we did for each of the imagery sources in this notebook.

The imagery sources to add are:

ASTER: ee.ImageCollection("ASTER/AST_L1T_003"), hint: request 2019 april to october season
https://developers.google.com/earth-engine/datasets/catalog/ASTER_AST_L1T_003

Sentinel-2: ee.ImageCollection('COPERNICUS/S2')
https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

Sentinel-3: ee.ImageCollection("COPERNICUS/S3/OLCI") hint: this satellite behaves like MODIS
https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S3_OLCI

PROBA-V: ee.ImageCollection("VITO/PROBAV/C1/S1_TOC_100M") hint: request 2019 gorwing season
https://developers.google.com/earth-engine/datasets/catalog/VITO_PROBAV_C1_S1_TOC_100M

VIIRS: ee.ImageCollection("NOAA/VIIRS/001/VNP09GA") hint: this satellite behaves like MODIS
https://developers.google.com/earth-engine/datasets/catalog/NOAA_VIIRS_001_VNP09GA

BONUS: HYPERSPECTRAL SENSOR
EO-1 Hyperion Hyperspectral Imager: ee.ImageCollection("EO1/HYPERION") 
hint: this satellite has 220 bands, choose between 2001 and 2017 to get an image for Cache Valley
https://developers.google.com/earth-engine/datasets/catalog/EO1_HYPERION


Happy coding!